# Исследование надёжности заёмщиков

## Описание проекта:

Заказчик — кредитный отдел банка. Цель проекта - изучение факторов (возраст, пол, уровень дохода и т.д.) влияющих на погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Содержание:
* [Общая информация о данных.](#section_1)
* [Предобработка данных:](#section_2_1)
    * [Замена типа данных.](#section_2_2)
    * [Обработка дубликатов.](#section_2_3)
    * [Лемматизация.](#section_2_4)
    * [Категоризация данных.](#section_2_5)
* [Анализ данных:](#section_3_1)
    * [Зависимость между наличием детей и возвратом кредита в срок.](#section_3_1)
    * [Зависимость между семейным положением и возвратом кредита в срок.](#section_3_2)
    * [Зависимость между уровнем дохода и возвратом кредита в срок.](#section_3_3)
    * [Как разные цели кредита влияют на его возврат в срок?](#section_3_4)
* [Общий вывод](#section_4)


## Общая информация о данных.<a class="anchor" id="section_1">

In [17]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод <a class="anchor" id="conclusion_1">
Файл содержит 2525 строк и 12 столбцов. 
    
Типы данных в столбцах: 
* вещественные числа- 2, 
* целочисленные значения -5, 
* объекты -5.

В таблице представлены следующая информация о заемщиках:
    
* количество детей в семье
* общий трудовой стаж в днях
* возраст клиента в годах
* уровень образования клиента
* идентификатор уровня образования
* семейное положение
* идентификатор семейного положения
* пол клиента
* тип занятости
* имел ли задолженность по возврату кредитов
* ежемесячный доход
* цель получения кредита
    
Прежде чем приступать к анализу необходима тщательная предобработка т.к. данные содержат некорректные значения, что может повлиять на результаты исследования.

## Предобработка данных. <a class = "anchor" id ='section_2'>

### Обработка пропусков<a class = "anchor" id ='section_2_1'>

In [3]:
df['days_employed'] = df['days_employed'].fillna(0)

median_incomes = df.groupby('income_type')['total_income'].median().to_dict()

def filler_func(row):
    if pd.isna(row['total_income']):
         return median_incomes[row['income_type']]
    return row['total_income']

df['total_income'] = df.apply(filler_func, axis=1)

df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Пропущенные значения в столбце 'days_employed'ачения были заменены на "0". Значения в столбце 'total_income' были заменены на медиану от значений сгруппированных по типу занятости. Отсутсвующие значения могли быть обусловлены ошибками при вводе или нежеланием/невозможностью клиентов разглашать информацию. 

### Замена типа данных<a class = "anchor" id ='section_2_2'>

In [4]:
#для корректной работы с данными неободимо заменить типо float на int, 
#меняем данный сразу во всем столбце при помощи метода astype()

df['days_employed'] = abs(df['days_employed'].astype(int))
df['total_income'] =abs(df['total_income'].astype(int))

### Вывод

При знакомстве с данными были выявлены аномальные значения: отрицательный трудовой стаж, отрицательные значения в по столбце 'children'. Столбцы были обработаны функцией, заменяющей отрицательные значения на положительные. Так же дыли изменены вещественные значения на целочисленные.

Ошибки могли появится в процессе выгрузки данных, либо при ошибочном заполнеии полей при сборе.

### Обработка дубликатов<a class = "anchor" id ='section_2_3'>

In [5]:
# в столбце 'education' приводим значения к одному регисстру
# в данном случае нас интересуют полные дубликаты, поэтому используем метод drop_duplicates + reset_index для переиндексации строк

df['education'] = df['education'].str.lower()
df = df.drop_duplicates().reset_index(drop = True)


### Вывод

Посредствам применения метода drop_duplicates были удалены полные дубликаты строк во всем фрейме. Дубликаты могли появится на этапе копирования или переноса данных либо же данные о некоторых клиентах могли полностью совпасть.

### Лемматизация<a class = "anchor" id ='section_2_4'>

In [6]:
m = Mystem()

def lemmatisizer(row):
    return m.lemmatize(row['purpose'])

df['lemmatisized'] = df.apply(lemmatisizer, axis=1)


### Вывод

В результате лемматизации значений в столбце 'purpose' получаем новый столбец с леммами, необходимый для последующей категоризации.

## Категоризация <a class = "anchor" id ='section_2_5'>

In [7]:

def categorized(purpose):
    if 'автомобиль' in purpose or 'aвто' in purpose:
        return 'автомобиль'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'образование'in purpose:
        return 'образование'
    if 'недвижимость'in purpose or 'квартира'in purpose or 'дом' in purpose or 'жилье' in purpose:
        return 'недвижимость'
    return 'другое'

df['categorized_purpose'] = df['lemmatisized'].apply(categorized)

df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatisized,categorized_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


### Вывод

По результатам лемматизации была проведена категоризация объектов, необходимая для дальнейших расчетов.

## Анализ данных<a class = "anchor" id ='section_3_1'>

### - Есть ли зависимость между наличием детей и возвратом кредита в срок?<a class = "anchor" id ='section_3_2'>


In [8]:
#проводим категоризацию для удобства
def categorizing_children(row):
    if row == 0:
        return 'нет детей'
    if row > 0:
        return 'есть дети'
    return 'другое'
df['categorized_сhildren'] = df['children'].apply(categorizing_children)    

#составляем сводную таблицу                        
children_df = df.pivot_table(index = ['categorized_сhildren'], columns='debt', values ='children', aggfunc='count')

#заменяем названия колонок
children_df.set_axis(['нет задолженности','есть задолженность'], axis = 'columns',inplace = True)

#расчитывает долю
children_df['доля имевших задолженность'] = (children_df['есть задолженность'] * 100)/ (children_df['нет задолженности'] + children_df['есть задолженность'])
children_df['доля без задолженности'] = (children_df['нет задолженности'] * 100)/ (children_df['нет задолженности'] + children_df['есть задолженность'])

print('Доля имевших задолженность с детьми: {:.2f}'.format(children_df['доля имевших задолженность'][0]),'%')
print('Доля имевших задолженность без детей {:.2f}'.format(children_df['доля имевших задолженность'][1]),'%')

#расчитываем разницу между долями
difference_children = children_df['доля имевших задолженность'][0] - children_df['доля имевших задолженность'][1]

print('Люди без детей возвращают кредит в срок на {:.2f}'.format(difference_children),'% чаще.')
children_df



Доля имевших задолженность с детьми: 9.21 %
Доля имевших задолженность без детей 7.54 %
Люди без детей возвращают кредит в срок на 1.66 % чаще.


,нет задолженности,есть задолженность,доля имевших задолженность,доля без задолженности
categorized_сhildren,,,,
есть дети,6685,678,9.208203,90.791797
нет детей,13028,1063,7.543822,92.456178


### Вывод

В результате анализа было определено - клиенты, не имеющие детей, возвращат задолженность в срок на 1.67 % чаще.

### - Есть ли зависимость между семейным положением и возвратом кредита в срок? <a class = "anchor" id ='section_3_3'>

In [9]:
# расчет по аналогии с предыдущим
family_df = df.pivot_table(index = ['family_status'], columns = 'debt', values = 'family_status_id', aggfunc = 'count')
family_df.set_axis(['нет задолженности','есть задолженность'], axis = 'columns',inplace = True)
family_df['доля имевших задолженность'] = (family_df['есть задолженность'] * 100)/ (family_df['нет задолженности'] + family_df['есть задолженность'])
family_df['доля без задолженности'] = (family_df['нет задолженности'] * 100)/ (family_df['нет задолженности'] + family_df['есть задолженность'])

# сортируем для нагядности
family_df.sort_values(by = 'доля имевших задолженность', ascending = False)

,нет задолженности,есть задолженность,доля имевших задолженность,доля без задолженности
family_status,,,,
Не женат / не замужем,2536,274,9.750890,90.249110
гражданский брак,3763,388,9.347145,90.652855
женат / замужем,11408,931,7.545182,92.454818
в разводе,1110,85,7.112971,92.887029
вдовец / вдова,896,63,6.569343,93.430657


### Вывод

В результате анализа удалось выяснить, что максимальный процент клиентов имевших задолженность наблюдается в категории 'Не женат / не замужем' - 9.7%, далее, клиенты, состоящие в гражданском браке - 9.3%, женатые и замужние -7.5%. 

Сымыми надежными заемщиками являются находящиеся в разводе - 7.1% и вдовцы/вдовы - 6.5%.

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a class = "anchor" id ='section_3_4'>



In [10]:
# для выделения категорий некорректно просто целочисленно разделить максимальное сумму дохода на количествокатегорий
# поэтому шаг был выбран произвольно, ориентировался на медиану 

#print(df['total_income'].median())

def categorizing_total_income(row):
    if row <= 90000:
        return 'меньше 90к'
    if row > 90000 and row <= 120000:
        return 'от 90к до 120к'
    if row > 120000 and row <= 145000:
        return 'от 120к до 145к'
    if row > 145000 and row <= 185000:
        return 'от 145к до 185к'
    if row > 185000 and row <= 235000:
        return 'от 185к до 235к'
    if row > 235000:
        return 'больше 235к'
    return 'другое'

df['categorized_income'] = df['total_income'].apply(categorizing_total_income)

total_income_df = df.pivot_table(index = ['categorized_income'], columns='debt', values ='total_income', aggfunc='count')
total_income_df.set_axis(['нет задолженности','есть задолженность'], axis = 'columns',inplace = True)
total_income_df['доля имевших задолженность'] = (total_income_df['есть задолженность'] * 100)/ (total_income_df['нет задолженности'] + total_income_df['есть задолженность'])
total_income_df['доля без задолженности'] = (total_income_df['нет задолженности'] * 100)/ (total_income_df['нет задолженности'] + total_income_df['есть задолженность'])

total_income_df.sort_values(by = 'доля имевших задолженность', ascending = False)




,нет задолженности,есть задолженность,доля имевших задолженность,доля без задолженности
categorized_income,,,,
от 120к до 145к,3548,351,9.002308,90.997692
от 90к до 120к,3559,324,8.344064,91.655936
от 145к до 185к,3879,351,8.297872,91.702128
от 185к до 235к,2573,227,8.107143,91.892857
меньше 90к,3086,262,7.825568,92.174432
больше 235к,3068,226,6.860959,93.139041


### Вывод

В результате анализа удалось определить, что самая высокая доля клиентов - 9% имеют общий доход в интерваое от 120 до 145 тысяч рублей. Наиболее надежными заемщиками являются клиенты с оченьвысоким доходом - больше 235 тысяч или же, напротив, с низким - до 90 тысяч рублей.

### - Как разные цели кредита влияют на его возврат в срок?<a class = "anchor" id ='section_3_5'>

In [11]:
df_purpose = df.pivot_table(index = ['categorized_purpose'], columns='debt', values ='purpose', aggfunc='count')
df_purpose.set_axis(['нет задолженности','есть задолженность'], axis = 'columns',inplace = True)
df_purpose['доля имевших задолженность'] = (df_purpose['есть задолженность'] * 100)/ (df_purpose['нет задолженности'] + df_purpose['есть задолженность'])
df_purpose['доля без задолженности'] = (df_purpose['нет задолженности'] * 100)/ (df_purpose['нет задолженности'] + df_purpose['есть задолженность'])
df_purpose.sort_values(by = 'доля имевших задолженность', ascending = False)



,нет задолженности,есть задолженность,доля имевших задолженность,доля без задолженности
categorized_purpose,,,,
автомобиль,3903,403,9.359034,90.640966
образование,3643,370,9.220035,90.779965
свадьба,2138,186,8.003442,91.996558
недвижимость,10029,782,7.233373,92.766627


### Вывод

В зависимости от цели кредита самыми надежными заемщиками являются клиенты, бравшие заем на недвижимость - 7.2 % не имевших задолженность. 
Самая большая доля клиентов имевших задолженность приходится на категорию обозначевших целью кредита авто - 9.3%.

## Общий вывод <a class = "anchor" id ='section_4'>

В результате исследования удалось обнаружить следующие зависимости:

* Клиенты, не имеющие детей, возвращают задолженность в срок на 1.67 % чаще.

* Самая высокая доля клиентов не вернувших займ в срок- 9% имеют общий доход в интервале от 120 до 145 тысяч рублей. Большая доля клиентов не имевших просроченной задолженности получают доход либо больше 235 тысяч либо же, до 90 тысяч рублей.

* В зависимости от семейного положения прослеживается следующая корреляция: максимальный процент клиентов имевших задолженность наблюдается в категории 'Не женат / не замужем' - 9.7%, далее, клиенты, состоящие в гражданском браке - 9.3%, женатые и замужние -7.5%. Меньший процент клиентов н имевших задолженность находядятся в разводе - 7.1% либо являются вдовцами/вдовами - 6.5%.

* В зависимости от цели кредита меньшая доля клиентов с непогашенной ранее задолженностью - бравшие заем на недвижимость - 7.2 % не имевших задолженность. Самая большая доля клиентов имевших задолженность приходится на категорию обозначивших целью кредита 'авто' - 9.3%.

Опираясь на результаты исследования можно утверждать, что самыми надежными заемщиками являются люди не имеющие детей, бывшие в браке, определяющие целью кредита недвижимость и получающие суммарный доход либо более 235, либо менее 90 тыс. рублей.

Нежелательными заемщиками являются клиенты с детьми,и не состоящие в браке, определяющие цель кредита как образование' или 'авто' и получающие суммарный доход в интервале от 120 до 145 тыс. рублей.
